<center><h2><b>CryptoSight</b></h2></center>
<center><i>Il lungo e tortuoso viaggio nelle inter-compatibilità</i></center>
<center><img src="./docs/images/dashboard.png" alt="CryptoSight" width="700"/></center>

# Project Goal 💻

CryptoSight è il progetto sviluppato per il corso di *Technologies for Advanced Programming* dell'UniCT. 

L'obbiettivo principale del progetto è mettere in pratica gli strumenti spiegati nel corso al fine di creare una pipeline completa di elaborazione, che vada dalla *Data Ingestion* alla visualizzazione dei dati per l'utente finale attraverso *OpenSearch Dashboard*

CryptoSight si incentra sul tracking di alcune statistiche legato al mondo delle cryptomonete: il cambio, volume, unità di circolazione, capitalizzazione di mercato di una piccola selezione di monete.

Le monete tracciate sono: Bitcoin, Ethereum, XRP, Cardano, Binance Coin, Dogecoin.

La pipeline presenta una piccola applicazione di *Machine Learning* riguardante la *previsione* del cambio delle monete.

## La pipeline 👀
<center><img src="./docs/images/pipeline.png" alt="pipeline"/></center>

<center><b>L'idea di usare OpenSearch mi è costata la sanità mentale, ma è andata</b></center>

## Data Source ⚠️


Per la realizzazione del progetto si ci è basato su una piattaforma di tracking denominata *CoinGecko*. 
<center><img src="./docs/images/coingecko.jpg" alt="I am poor" width="300"/></center>

La piattaforma offre una serie di piani... molto costosi. Mi sono limitato all'utilizzo del piano gratuito che offre un massimale di 50 richieste/minuto.

<center><img src="./docs/images/expensive.jpg" alt="I am poor" width="400"/></center>

## Data Ingestion 🔧 
<center><img src="./docs/images/logstash.png" alt="Logstash" width="400"/></center>
Con il termine *Data Ingestion* si intende il processo con il quale vengono acquisiti e importati i dati per uso immediato o per archiviazione in un database.

Per il progetto si è scelto di utilizzare Logstash per la sua facilità di utilizzo e integrazione con gli altri componenti della pipeline. <i>In particolare l'integrazione con OpenSearch <b>dovrebbe</b> essere garantita da Logstash FOSS con OS Plugin... <b>ma la vita non è mai semplice.</b></i>

Logstash sfrutta il plugin *http_poller* come canale input per eseguire uno *streaming* (fetch real-time) di *dati strutturati* (contenuto in un comunissimo formato JSON) verso Kafka attraverso il plugin di output dedicato

## Logstash in dettaglio

L'http_poller esegue periodicamente la chiamata al seguente indirizzo:

> https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids=bitcoin%2Cethereum%2Cdogecoin%2Ccardano%2Ctether%2Cbinancecoin%2Cripple&order=market_cap_desc&per_page=100&page=1&sparkline=false

Questa chiamata ritorna un *JSONArray* che mediante l'utilizzo del *codec json* viene spacchettato in *singoli messaggi* del topic di Kafka

## Data Streaming e Stream Processing 🔨

<center><img src="./docs/images/kafka.jpeg" alt="Kafka" width="400"/></center>

Basandoci sulla filosofia MOM (*Middleware orientato ai messaggi*) e lo *Stream Processing*, si impiega Kafka e uno KafkaStream per elaborare i messaggi *"raw"* ricevuto da Logstash. I dati vengono *puliti da informazioni non utilizzate e viene immesso un fattore di variazione per scopi didattici*.

Si è preferito eseguito la pulizia dei dati su KafkaStream anzichè su Logstash, mediante la funzionalità *filter*, poichè per possibili ampliazioni future basterà metter mano solamente sul KafkaStream. Ciò, in aggiunta alla persistenza dello storage di Kafka, ci permette di poter variare l'elaborazione dei dati senza dover rinunciare a tutti i dati raccolti fino a quel momento.

## Inizia la sofferenza 😢

<center><img src="./docs/images/esos.jpg" alt="ESOS" width="400"/></center>

Nel corso di TAP abbiamo visto e usato *ElasticSearch*, un motore di analisi e ricerca dei dati, e *Kibana*, un applicazione front-end grauito che collegandosi a ElasticSearch permette l'utilizzo dei dati sotto forma di grafici o altre funzionalità più complesse.

Nel mentre però... **ElasticSearch ha fatto l'avido cambiando i termini della propria licenza** (*per maggiori informazioni, leggersi le slides sull'argomento presenti sul mio GitHub*), ciò ha portato alla nascità di un fork di entrambe le piattaforma sopracitate:

- ElasticSearch => OpenSearch
- Kibana => OpenSearch-Dashboards

**Credo che il meme indichi chiaramente la strada ("strada", l'avete capita?) che ho intrapreso... non l'avessi mai fatto!**

## Kafka to OpenSearch 📦

<center><img src="./docs/images/opensearch.jpg" alt="OS" width="400"/></center>

OpenSearch ha rivoluzionato un pochetto le modalità di comunicazione con gli altri componenti. In particoliamo evidenzio:
- OpenSearch nella comunicazione utilizza il proprio versioning, che parte da 1.0, anziché quello di Elastic, che attualmente sta alla 7.15;
- La comunicazione HTTP anomina è stata deprecata, ora è necessario connettersi tramite HTTPS (quindi avere a che fare con certificati SSL) e fornire delle credenziali;
- La porta del performance analyzer è stata cambiata;
- Non presenta il plugin per le operazioni senza autenticazione;

### Quindi?

## Le scelte, quelle importanti! 😶

<center><img src="./docs/images/kafkaingestion.jpg" alt="kafka is hungry" width="300"/></center>

Guardando le scelte implementative degli altri colleghi ho visto una tendenza ad usare:
- Kafka Connect to ElasticSearch
- Un Kafka Consumer ad hoc

Dati i problemi sopracitati, OpenSearch è venuto incontro alla community fornendo una distribuzione di *Logstash FOSS con un plugin output dedicato a OpenSearch*.

**La scelta finale è stata immediata.**

## I primi grafici! 📈

<center><img src="./docs/images/opendashboards.jpg" alt="I need data!" width="300"/></center>

Possiamo loggare su OpenSearch Dashboards per creare grafici!

<center><img src="./docs/images/marketcap.png" alt="Circles" width="300"/> <img src="./docs/images/volume.png" alt="Squares" width="300"/></center>

## Siamo a buon punto! ☕️

<center><img src="./docs/images/prefail.jpg" alt="I need data!" width="500"/></center>

## Spark e Machine Learning ✨

Siamo nel 2021, il termine "Machine Learning" fa gola a tutti... infiliamo un pò di ML nella pipeline!

Come già detto precedentemente, la pipeline svolte un'operazione di machine learning per *l'arricchimento dei dati attraverso una predizione del cambio futuro*

Viene attuato uno Spark Streaming con una finestra di 1 minuti per predire i successivi valori.

### Dove sta il problema?

## Spark & OpenSearch ⚠️

<center><img src="./docs/images/sparkerror.jpg" alt="Ready"/></center>

I cambiamenti di OpenSearch hanno creato non pochi grattacapi per quanto rigurada la comunicazione fra PySpark e OpenSearch.

- L'obbligo di una connessione HTTPS ha richieste molti più parametri del normale ElasticSearch.
- L'override del versioning causava problemi di ambiguità nel comportamento di Spark.

Per l'ultimo problema OpenSearch ha fornito una soluzione.

> compatibility.override_main_response_version=true


### E' fatta allora?

<center><img src="./docs/images/versioning.jpg" alt="Home"/></center>


## Cos'è successo?

Abilitando la **compatibility mode** di OpenSearch si è rotto il supporto per **Logstash FOSS con plugin OS*

<center><img src="./docs/images/compatibility.png" alt="Home"/></center>
Ma ciò non era stato detto...

## Let's start!

> docker-compose up -d

# Thank you for your attention

